In [ ]:
import awkward as ak
import numpy as np

import hepfile

import uproot

import time

print(ak.__version__)
print(uproot.__version__)

In [ ]:
# Down load a file for us to play with
!curl http://opendata.cern.ch/record/12361/files/SMHiggsToZZTo4L.root --output SMHiggsToZZTo4L.root

In [ ]:
# This is all for demonstration purposes, to show people how this type of
# writing could be done. 
# But of course people could just create their own awkward arrays.

f = uproot.open('SMHiggsToZZTo4L.root')

events = f['Events']

events

In [ ]:
# What's in this file? 
events.keys()

In [ ]:
# Let's check out these types and their sizes

print(type(events['Muon_pt'].array()))
print()

# Number of events
print(ak.num(events['Muon_pt'].array(),axis=0))

# Number of muons in each event
print(ak.num(events['Muon_pt'].array(),axis=1))

# Number of muons in total
print(ak.sum(ak.num(events['Muon_pt'].array(),axis=1)))

In [ ]:
# While not all the entries in the ROOT file naturally lend themselves to 
# group/dataset breakdowns, some do. Let's find those "automatically", 
# just to make it easier to write them to the hepfile.

# Find groups
def make_groups_and_datasets(fields):
    
    groups = {}
    
    for field in fields:
        if field.find('_')>=0:
            
            # Do this in case there is more than one underscore
            idx = field.find('_')
            
            #print(field)
            grp = field[0:idx]
            dset = field[idx+1:]
            
            if grp not in groups.keys():
                groups[grp] = [[field,dset]]
            else:
                groups[grp].append([field,dset])
    
    return groups


############################################################

groupings = make_groups_and_datasets(events.keys())

# Groupings gives us a nice mapping of the names from the ROOT file
# to how we're going to store them in our hepfile as 
# group/datasets

print(groupings)
print()
print(groupings['Muon'])

In [ ]:
# There are some others. THese will be SINGLETONS that we pass in separately.
'run',
'luminosityBlock',
'event',

print(events['run'].array())
print(events['luminosityBlock'].array())
print(events['event'].array())

In [ ]:
def _get_awkward_type(ak_array: ak.Record) -> type:
    try:
        if isinstance(ak_array[0], (ak.Record, ak.Array)):
            arr = ak_array
            #import pdb; pdb.set_trace()
            
            #print(ak_array.type.content)
            
            type_str = ak_array.type.content            
            
            if isinstance(type_str, ak.types.NumpyType):
                dtype = type_str.primitive
            else:
                dtype = str(type_str).rsplit("*", maxsplit=1)[-1].strip()
            
        else:
            arr = np.array(ak_array)
            dtype = arr.dtype

        if dtype == "string":
            dtype = np.dtype("<U1")

        np_dtype = np.dtype(dtype)
        if np_dtype.char == "U":
            np_dtype = str

    except Exception as exc:
        raise IOError("Cannot convert input value to a numpy data type!") from exc

    return np_dtype

# Try it out
print(_get_awkward_type(events['Muon_pt'].array()))
print(_get_awkward_type(events['MET_pt'].array()))
print(_get_awkward_type(events['run'].array()))

In [ ]:
# I changed the name of this function to pack multiple awkward arrays.
# I agree with you that we should probably also have a function that packs
# single awkward arrays that gets called by this function.

# d = data dictionary, maybe we have a version of this that automatically
# creates the dictionary and returns it?

# arr = dictionary of awkward arrays and keys to use as names for the dataset

# group_name = group the dataset will belong to

# Right now, I am creating a counter "n{group_name}", is that what we want?
# Or should we allow the user to pass in the counter name, like when
# create_group is called?
#
# Right now, it creates the group each time this function is called
# We should have it check to see if the group is already there and if so, 
# don't add it again.

def pack_multiple_awkward_arrays(d, arr, group_name=None):
    
    # If the user passed in a group, then the datasets will 
    # be under that group

    # If no groupname is passed in, we will assume these are singleton datasets    
    if group_name is not None:
        counter = f"n{group_name}"
        d['_GROUPS_'][group_name] = [counter]

        # We will use this name for the counter later
        counter = f"{group_name}/n{group_name}"
        d['_MAP_DATASETS_TO_DATA_TYPES_'][counter] = int

        d['_MAP_DATASETS_TO_COUNTERS_'][group_name] = counter
        d['_LIST_OF_COUNTERS_'].append(counter)
    else:
        counter = '_SINGLETONS_GROUP_/COUNTER'
        

    # Loop over the dictionary that is passed in
    for field in arr.keys():
        # build a name for the hepfile entry
        dataset_name = f"{group_name}/{field}"
        
        # If there is no group name, it is a SINGLETON
        if group_name is None:
            dataset_name = f"{field}"
        
        #print(field)
        
        print(dataset_name)
        
        # Get the values
        x = arr[field]
        
        # For debugging
        #print(f"\t{v}   {x.ndim}")
        
        # Tells us if this is jagged or not
        dtype = _get_awkward_type(x)
        
        if x.ndim==1:

            # Get the datatpe before we flatten it
            dtype = _get_awkward_type(x)
            
            x = ak.to_numpy(x)
            num = np.ones(len(x),dtype=int) # This is repeated, should we only do it once?
        
        else:

            # Get the datatpe before we flatten it
            dtype = _get_awkward_type(x)
            
            # This saves the counter as int64, taking up a bit more space
            # Probably minimal though. 
            #num = ak.num(x)
            # This saves the counter as int32
            num = ak.to_numpy(ak.num(x)).astype(np.int32)

            x = ak.flatten(x).to_numpy()
            
        d[dataset_name] = x
        
        #print(dtype_dict[dtype])
        
        # Not a SINGLETON, the user has passed in a groupname
        if group_name is not None:
            d['_MAP_DATASETS_TO_DATA_TYPES_'][dataset_name] = dtype
            d['_MAP_DATASETS_TO_COUNTERS_'][dataset_name] = counter
            d['_GROUPS_'][group_name].append(field)
        # If it is a SINGLETON
        else:
            d['_MAP_DATASETS_TO_DATA_TYPES_'][dataset_name] = dtype
            d['_MAP_DATASETS_TO_COUNTERS_'][dataset_name] = '_SINGLETONS_GROUP_/COUNTER'
            d['_GROUPS_']['_SINGLETONS_GROUP_'].append(dataset_name)        

    d[counter] = num
    # We don't need to return the dictionary because in python
    # dictionaries are mutable. The dictionary in the function points to 
    # the same object outside of the function.
###############################################################################


# Here's how we call it. Though maybe we have an option to have
# the data dictionary created inside the function? 

# Initialize the data dictionary
data = hepfile.initialize()

# Pack these groups of awkward arrays

# This is what it would look like "by hand"
# A dictionary with the name of the dataset as it is to appear inside the hepfile
# and then the actual awkward array (not just the Branch object returned by uproot)
#ak_arrays = {'pt': events['Muon_pt'].array(), 'eta': events['Muon_eta'].array(), 'phi':events['Muon_phi'].array(), }

# Here I'm packing all the data that are groups/datasets
for groups_to_write in ['Muon', 'Electron', 'MET', 'PV']:
    ak_arrays = {}
    for grouping in groupings[groups_to_write]:
        #print(grouping)
        ak_arrays[grouping[1]] = events[grouping[0]].array()
    
    # For debugging
    #print(ak_arrays)
    
    pack_multiple_awkward_arrays(data, ak_arrays, group_name=groups_to_write)

# Now the SINGLETONS
ak_arrays = {"run":events['run'].array(), \
             "luminosityBlock":events['luminosityBlock'].array(), \
             "event":events['event'].array()}

# Note that there is no group name passed in. 
pack_multiple_awkward_arrays(data, ak_arrays)
    

In [ ]:
# Uncomment if you want to see what the data dictionary looks like
#data

print(data['_GROUPS_']['_SINGLETONS_GROUP_'])
print()

print(data['_MAP_DATASETS_TO_COUNTERS_'])
print()

data.keys()

In [ ]:
# Write it!

# Try it with no compression
start = time.time()
hepfile.write_to_file('awkward_write_test.h5', data, verbose=False)
dt_no_compression = time.time() - start

# Try it with compression!
start = time.time()
hepfile.write_to_file('awkward_write_test_COMP_gzip_OPT_9.h5', data, verbose=False, comp_type="gzip", comp_opts=9)
dt_with_compression = time.time() - start

print()
print()
print(f"Time to write uncompressed: {dt_no_compression}")
print(f"Time to write   compressed: {dt_with_compression}")

print()
print()

# Check out the sizes of the files!

!ls -ltr SMHiggsToZZTo4L.root
!ls -ltr awkward_write_test.h5
!ls -ltr awkward_write_test_COMP_gzip_OPT_9.h5

In [ ]:
data,bucket = hepfile.load('awkward_write_test_COMP_gzip_OPT_9.h5')

#data,bucket = hepfile.load('awkward_write_test.h5')

In [ ]:
data['Muon/nMuon']

In [ ]:
print(type(data['Muon/pt']))
print(type(data['Muon/pt'][0]))

In [ ]:
type(data['Muon/pt'].astype(np.float32)[0])

In [ ]:
data['run']

In [ ]:
data['_MAP_DATASETS_TO_DATA_TYPES_']

In [ ]:
data['_GROUPS_']

In [ ]:
data.keys()

In [ ]:
x = ak.to_numpy(ak.flatten(events['Muon_pt'].array()))

type(x)
type(x[0])

In [ ]:
x = events['Muon_pt'].array()

xnum = ak.num(x)

print(type(xnum))
print(type(xnum[0]))

print(xnum)

print(xnum[0])

print(type(ak.to_numpy(xnum)[0]))
print(type(ak.to_numpy(xnum).astype(np.int32)[0]))

# Scratch code

Just a bunch of test code when I was trying to figure this all out. 

In [ ]:
d = {}
groups_to_datasets = {}

counters = []

for field in events.fields:
    
    print(field)
    
    d[field] = []
    groups_to_datasets[field] = []
    
    counters.append(f'n{field}')
    
    for v in events[field].fields:
        groups_to_datasets[field].append(v)
        
        key = f"{field}/{v}"
        
        x = events[field][v]
        
        #print(v)
        
        print(f"\t{v}   {x.ndim}")
        
        if x.ndim==1:
            dtype = x.layout.format
            x = ak.to_numpy(x)

        else:
            dtype = x.layout.content.format
            x = ak.flatten(x).to_numpy()


        d[key] = x

In [ ]:
!ls -ltr

In [ ]:
ak.num(events['Muon']['pt'])

In [ ]:
events.luminosityBlock

In [ ]:
d

In [ ]:
x = events['MET']['pt']

print(x.ndim)
print(events['Muon']['pt'].ndim)
print(events['MET']['pt'].ndim)

In [ ]:
#x.layout
layout = events['Electron']['pt'].layout

In [ ]:
layout.content.format

In [ ]:
x = events['MET']['pt']

x.layout.format

In [ ]:
x = events['MET']['pt']

x.layout

In [ ]:
x = events['MET']['pt']

try:
    x = ak.flatten(x)
except:
    1
x = ak.to_numpy(x)

print(type(x))

# Testing out the loop way

In [ ]:
data = hepfile.initialize()
data

In [ ]:
hepfile.create_group(data,group_name='muon',counter='nmuon')
hepfile.create_dataset(data,group='muon',dset_name=['px','py','pz'],dtype=float)

hepfile.create_dataset(data,dset_name=['luminosity_block'],dtype=int)

bucket = hepfile.create_single_bucket(data)

In [ ]:
data

In [ ]:
nevents = 100

for i in range(nevents):
    nmuon = np.random.randint(0,5)
    bucket['muon/nmuon'] = nmuon
    bucket['muon/px'] = np.random.random(nmuon).tolist()
    bucket['muon/py'] = np.random.random(nmuon).tolist()
    bucket['muon/pz'] = np.random.random(nmuon).tolist()
    
    bucket['luminosity_block'] = np.random.randint(100,10000)
    
    hepfile.pack(data,bucket)

hepfile.write_to_file('awkward_write_test_LOOP_FILL.h5', data, verbose=True)
 

In [ ]:
data,bucket = hepfile.load('awkward_write_test_LOOP_FILL.h5')

In [ ]:
data['luminosity_block']